In [2]:
import xarray as xr
import fsspec
from pathlib import Path
from dask.diagnostics import ProgressBar
import pandas as pd
import numpy as np
import scipy.spatial

In [ ]:
fs = fsspec.filesystem('gs', token='anon')
fs.ls('gs://gcp-public-data-arco-era5/co')

reanalysis = xr.open_zarr(
    fs.get_mapper('gs://gcp-public-data-arco-era5/co/single-level-reanalysis.zarr'),
    consolidated=True,
)

print(reanalysis.chunks)

/tmp/ipykernel_1080261/2194816378.py:4: FutureWarning: In a future version, xarray will not decode the variable 'step' into a timedelta64 dtype based on the presence of a timedelta-like 'units' attribute by default. Instead it will rely on the presence of a timedelta64 'dtype' attribute, which is now xarray's default way of encoding timedelta64 values.
To continue decoding into a timedelta64 dtype, either set `decode_timedelta=True` when opening this dataset, or add the attribute `dtype='timedelta64[ns]'` to this variable on disk.
To opt-in to future behavior, set `decode_timedelta=False`.
  reanalysis = xr.open_zarr(


<xarray.Dataset> Size: 1PB
Dimensions:     (time: 1323648, hybrid: 137, values: 410240)
Coordinates:
  * time        (time) datetime64[ns] 11MB 1900-01-01 ... 2050-12-31T23:00:00
  * hybrid      (hybrid) float64 1kB 1.0 2.0 3.0 4.0 ... 134.0 135.0 136.0 137.0
    step        timedelta64[ns] 8B ...
    valid_time  (time) datetime64[ns] 11MB dask.array<chunksize=(1323648,), meta=np.ndarray>
Dimensions without coordinates: values
Data variables:
    d           (time, hybrid, values) float32 298TB dask.array<chunksize=(1, 1, 410240), meta=np.ndarray>
    t           (time, hybrid, values) float32 298TB dask.array<chunksize=(1, 1, 410240), meta=np.ndarray>
    vo          (time, hybrid, values) float32 298TB dask.array<chunksize=(1, 1, 410240), meta=np.ndarray>
    w           (time, hybrid, values) float32 298TB dask.array<chunksize=(1, 1, 410240), meta=np.ndarray>
Attributes: (12/14)
    Conventions:               CF-1.7
    GRIB_centre:               ecmf
    GRIB_centreDescription:    

In [3]:
def build_regridder_from_reduced_da(da, out_lon, out_lat):
    """
    da: xarray DataArray with dims (..., values) and coords latitude(values), longitude(values)
        (like ERA5 reduced Gaussian exposed by cfgrib)
    out_lon: 1D array of target longitudes (e.g. 0..360)
    out_lat: 1D array of target latitudes  (e.g. -90..90)

    Returns:
      regrid(values_1d) -> 2D array with shape (len(out_lat), len(out_lon))
    """
    # Source points (1D, aligned with 'values' dim)
    src_lon = np.asarray(da.longitude.values, dtype=np.float64)
    src_lat = np.asarray(da.latitude.values, dtype=np.float64)

    # Mirror lon==0 -> 360 to avoid seam artifacts near the dateline
    # (works when you target [0, 360], inclusive)
    m = np.isclose(src_lon, 0.0)
    src_lon2 = np.concatenate([src_lon, src_lon[m] + 360.0])
    src_lat2 = np.concatenate([src_lat, src_lat[m]])

    # Delaunay triangulation in (lon, lat)
    tri = scipy.spatial.Delaunay(np.stack([src_lon2, src_lat2], axis=1))

    # Target mesh points: (N, 2) list of lon/lat pairs
    # We'll output as (lat, lon) for typical image/array conventions.
    Lon2d, Lat2d = np.meshgrid(out_lon, out_lat, indexing="xy")
    mesh = np.stack([Lon2d.ravel(), Lat2d.ravel()], axis=1)

    # Precompute simplex + barycentric weights once (this is the big win)
    simplex = tri.find_simplex(mesh)
    X = mesh
    T = tri.transform

    ndim = 2
    T_inv = T[simplex, :ndim, :]              # (N,2,2)
    r = T[simplex, ndim, :]                   # (N,2)
    c = np.einsum("nij,nj->ni", T_inv, X - r) # (N,2)
    w = np.concatenate([c, 1.0 - c.sum(axis=1, keepdims=True)], axis=1)  # (N,3)

    verts = tri.simplices[simplex]            # (N,3)
    valid = simplex != -1

    nlat = len(out_lat)
    nlon = len(out_lon)

    def regrid(values_1d):
        v = np.asarray(values_1d, dtype=np.float32)
        # apply the same mirror as we did for coords
        v2 = np.concatenate([v, v[m]])

        out = np.full(mesh.shape[0], np.nan, dtype=np.float32)
        vv = v2[verts[valid]]                 # (Nv,3)
        out[valid] = np.einsum("ni,ni->n", vv, w[valid]).astype(np.float32)

        return out.reshape(nlat, nlon)        # (lat, lon)

    return regrid

In [4]:
# target regular grid (0.25°)
out_lon = np.linspace(0, 360, 360*4 + 1)      # includes 360
out_lat = np.linspace(-90, 90, 180*4 + 1)

# build once from any one slice (no time slice needed if coords are time-invariant)
sample = reanalysis["t2m"].isel(time=0)
regrid = build_regridder_from_reduced_da(sample, out_lon, out_lat)

In [5]:
for t in pd.date_range("2020-01-01T00:00:00", periods=24, freq="H"):
    one_hour = reanalysis["t2m"].sel(time=t)
    arr_1d = one_hour.values                  # (542080,)
    arr_ll = regrid(arr_1d)                   # (721, 1441) for 0.25° with endpoints
    print(t, arr_1d.shape, arr_ll.shape)

/tmp/ipykernel_1059544/318842967.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  for t in pd.date_range("2020-01-01T00:00:00", periods=24, freq="H"):


2020-01-01 00:00:00 (542080,) (721, 1441)
2020-01-01 01:00:00 (542080,) (721, 1441)
2020-01-01 02:00:00 (542080,) (721, 1441)
2020-01-01 03:00:00 (542080,) (721, 1441)
2020-01-01 04:00:00 (542080,) (721, 1441)
2020-01-01 05:00:00 (542080,) (721, 1441)
2020-01-01 06:00:00 (542080,) (721, 1441)
2020-01-01 07:00:00 (542080,) (721, 1441)
2020-01-01 08:00:00 (542080,) (721, 1441)
2020-01-01 09:00:00 (542080,) (721, 1441)
2020-01-01 10:00:00 (542080,) (721, 1441)
2020-01-01 11:00:00 (542080,) (721, 1441)
2020-01-01 12:00:00 (542080,) (721, 1441)
2020-01-01 13:00:00 (542080,) (721, 1441)
2020-01-01 14:00:00 (542080,) (721, 1441)
2020-01-01 15:00:00 (542080,) (721, 1441)
2020-01-01 16:00:00 (542080,) (721, 1441)
2020-01-01 17:00:00 (542080,) (721, 1441)
2020-01-01 18:00:00 (542080,) (721, 1441)
2020-01-01 19:00:00 (542080,) (721, 1441)
2020-01-01 20:00:00 (542080,) (721, 1441)
2020-01-01 21:00:00 (542080,) (721, 1441)
2020-01-01 22:00:00 (542080,) (721, 1441)
2020-01-01 23:00:00 (542080,) (721

In [6]:
with ProgressBar():
    reanalysis["t2m"].sel(
        time=slice("2020-01-01T00:00:00", "2020-01-07T23:00:00")
    ).to_netcdf(
        "./test_outputs/t2m_2020_H1_1.nc",
        compute=True
    )


[########################################] | 100% Completed | 18.46 s
